In [ ]:
#pip install -U langchain langchain-mistralai FastAPI langserve langgraph sse_starlette nest-asyncio pyngrok uvicorn

# First LangChain-based Project - An LLM application that accepts a message and translates it into a target language
The project is from the book "Generative AI with Python and PyTorch: Navigating the AI frontier with LLMs, Stable Diffusion, and next-gen AI applications 2nd ed. Edition"

In [ ]:
import os

os.environ["LANGCHAIN_TRACING"]="true"
os.environ["LANGCHAIN_ENDPOINT"]= "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"]= "***"
os.environ["MISTRAL_API_KEY"] = "***"

from langchain_mistralai import ChatMistralAI

model = ChatMistralAI(model="mistral-large-latest")


In [2]:
from langchain_core.messages import HumanMessage, SystemMessage

'''messages = [
    SystemMessage(content="Translate the following from English into Arabic"),
    HumanMessage(content="hi!"),
]

model.invoke(messages)'''

'messages = [\n    SystemMessage(content="Translate the following from English into Arabic"),\n    HumanMessage(content="hi!"),\n]\n\nmodel.invoke(messages)'

--------------------------------------------------------------------------------------------------------------
This one of the examples used in the LangChain documentation, and we’ll build on this example by deploying the application we build using ngrok so that we can access a web application running in Collab asynchronously with the notebook process.

In [ ]:
!ngrok config add-authtoken ***

## Creating an LLM chain

We’ll need a few key ingredients here:
1. The <span style="color: magenta;">FastAPI framework</span> for web applications.
2. The <span style="color: magenta;">Uvicorn server</span> we’ll use to deploy the application once we’ve developed it.
3. We’ll expose the model to end users on a particular URL using the <span style="color: magenta;">add_routes function</span>.
4. We’ll also need the <span style="color: magenta;">PromptTemplate</span>, which allows us to specify how the model reads user input and what variables are expected.
5. The <span style="color: magenta;">StrOutputParser</span>, which converts “message” objects from LLMs to strings.
6. We’ll use the <span style="color: magenta;">nest_asynchio module</span> to run our FastAPI application inside the process thread of the Colab notebook.
7. We’ll also need the<span style="color: magenta;"> “chain” module</span> to connect different LangChain functions together in a series
of sequential steps.
8. Finally, we’ll also import the <span style="color: magenta;">Mistral model API</span> for this example but, in theory,
we could use any LLM in the LangChain library.

In [6]:
'''from pyngrok import ngrok

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
print(ngrok_tunnel.public_url+'/chain/playground')'''

"from pyngrok import ngrok\n\nngrok_tunnel = ngrok.connect(8000)\nprint('Public URL:', ngrok_tunnel.public_url)\nprint(ngrok_tunnel.public_url+'/chain/playground')"

In [7]:
from fastapi import FastAPI
import uvicorn
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_mistralai import ChatMistralAI
from langserve import add_routes
import nest_asyncio
from langchain_core.runnables import chain
nest_asyncio.apply()

In [8]:
system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages([
('system', system_template),
('user', '{text}')
])

In [9]:
model = ChatMistralAI(model="mistral-large-latest")

In [10]:
parser = StrOutputParser()

Now, we can create our chain. In the LangChain library, individual operations may be created in a <span style="color: magenta;">pipeline</span> or <span style="color: magenta;">chain</span> using the <span style="color: magenta;">pipe operator (|)</span>. The functions are executed from left to right, and the sequence can be saved to a variable:

In [11]:
chain = prompt_template | model | parser

### Creating the LLM application
If we want to host our chain in the cloud, we can create a simple FastAPI server:

In [12]:
app = FastAPI(
title="LangChain Server",
version="1.0",
description="A simple API server using LangChain's Runnable interfaces",
)

In [13]:
add_routes(
app,
chain,
path="/chain",
)

In [14]:
'''
# Original code:
from pyngrok import ngrok

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
print(ngrok_tunnel.public_url+'/chain/playground')
import nest_asyncio
nest_asyncio.apply()

import uvicorn

# Start FastAPI app directly in the notebook
uvicorn.run(app, host='0.0.0.0', port=8000)'''

# New code to run server in background thread
from pyngrok import ngrok
import threading
import uvicorn
import nest_asyncio

nest_asyncio.apply()

# Start ngrok tunnel first
ngrok_tunnel = ngrok.connect(8000)
print("Public URL:", ngrok_tunnel.public_url + "/chain/playground")

# Start FastAPI in background
def run_server():
    uvicorn.run(app, host='0.0.0.0', port=8000)

thread = threading.Thread(target=run_server, daemon=True)
thread.start()

Public URL: https://1695f666c44b.ngrok-free.app/chain/playground


INFO:     Started server process [8]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)



     __          ___      .__   __.   _______      _______. _______ .______     ____    ____  _______
    |  |        /   \     |  \ |  |  /  _____|    /       ||   ____||   _  \    \   \  /   / |   ____|
    |  |       /  ^  \    |   \|  | |  |  __     |   (----`|  |__   |  |_)  |    \   \/   /  |  |__
    |  |      /  /_\  \   |  . `  | |  | |_ |     \   \    |   __|  |      /      \      /   |   __|
    |  `----./  _____  \  |  |\   | |  |__| | .----)   |   |  |____ |  |\  \----.  \    /    |  |____
    |_______/__/     \__\ |__| \__|  \______| |_______/    |_______|| _| `._____|   \__/     |_______|
    
LANGSERVE: Playground for chain "/chain/" is live at:
LANGSERVE:  │
LANGSERVE:  └──> /chain/playground/
LANGSERVE:
LANGSERVE: See all available routes at /docs/
INFO:     188.161.36.190:0 - "GET /chain/playground HTTP/1.1" 307 Temporary Redirect
INFO:     188.161.36.190:0 - "GET /chain/playground/ HTTP/1.1" 200 OK
INFO:     188.161.36.190:0 - "GET /chain/playground/assets/index-40

Exception ignored in: <async_generator object HTTP11ConnectionByteStream.__aiter__ at 0x000001BE4C674240>
Traceback (most recent call last):
  File "C:\Users\LTC\anaconda3\Lib\site-packages\httpcore\_async\connection_pool.py", line 404, in __aiter__
    yield part
RuntimeError: async generator ignored GeneratorExit
